## Création du fichier résultat 

L'objectif est d'obtenir deux tableaux (un par feuille) qui regroupent: 
1- une vision globale sur les Objectifs de Développement Durable, 
2- une fiche indicateur dimensionnée sur un filtre dynamique obligatoire.

Import des librairies

Pour des raisons de reproductabilité, on installe la version 3.1.3 de openpyxl grâce à la commande suivante: 

In [1]:
pip install openpyxl==3.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install papermill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [papermill]
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import openpyxl
from openpyxl import Workbook   
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook 
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Font, Border, Side
from openpyxl.styles import Alignment
from openpyxl.chart.label import DataLabelList                                                                                                                                                      
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.worksheet.formula import ArrayFormula
from openpyxl.utils import quote_sheetname
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl.styles import Alignment, PatternFill
from openpyxl.worksheet.datavalidation import DataValidation
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import subprocess
import papermill as pm

print(openpyxl.__version__)

3.1.3


Vérification de l'existence du fichier Excel de résultat 

In [4]:
from openpyxl import Workbook

path_file = '../OUTPUT/TDB_ODD.xlsx'

# Si le fichier n'existe pas, on le crée grâce au script d'intégration: 
notebook_path = '../SCRIPTS/integration.ipynb'

if not os.path.exists(path_file):
    try:
        pm.execute_notebook(
            notebook_path,
            '../SCRIPTS/TDB_ODD.ipynb',
            kernel_name='python3'
        )
        print("Fichier créé grâce au notebook d'intégration.")
    except Exception as e:
        print("Erreur lors de l'exécution du notebook d'intégration.")
        print(e)
else:
    print("Fichier trouvé.")

Fichier trouvé.


In [5]:
# Chargement des données 
data = pd.read_csv("../DS_DEVDUR_data.csv", sep =";")
metadata = pd.read_csv("../DS_DEVDUR_metadata.csv", sep = ";")

## Tableau bilan 
Elements du tableau: 
    Comparer les quantités d'action, ou l'effort cumulé, sur chaque ODD et par type d'indicateur. 
    Identifier les sujets cibles principaux de développement durable. 
    Identifier les indicateurs les plus représentés (indifféremment de l'ODD concerné). 
    Identifier la répartition du public concerné (sex, âge).
    Identifier la répartition des actions par geolocalisation. 

In [6]:
# Chargement feuille bilan  
wb = load_workbook(path_file)

if 'BILAN' not in wb.sheetnames:
    wb.create_sheet('BILAN')
    wb.save(path_file)
    print("Feuille bilan créée.")
else: 
    print("Feuille bilan trouvée.")

if 'DATA_BILAN' not in wb.sheetnames:
    wb.create_sheet('DATA_BILAN')
    wb.save(path_file)
    print("Feuille data bilan créée.")
else: 
    print("Feuille data bilan trouvée.")

bilan = wb['BILAN']
data_bilan = wb['DATA_BILAN']

Feuille bilan trouvée.
Feuille data bilan trouvée.


In [7]:
# Calcul des longueurs pour chaque colonnes à exploiter 
list_col = ['ODD', 'INDICATEUR_DEVDUR', 'UNIT_MEASURE', 'OBS_STATUS','DEVDUR_COMPOSITE','TYPE_INDICATEUR_DEVDUR','TIME_PERIOD']
len_col ={}
for col in list_col:
    len_col[f"len_{col}"] = len(data[col].unique())+1 # on rajoute +1 car python commence à 0
len_col

{'len_ODD': 18,
 'len_INDICATEUR_DEVDUR': 176,
 'len_UNIT_MEASURE': 21,
 'len_OBS_STATUS': 7,
 'len_DEVDUR_COMPOSITE': 193,
 'len_TYPE_INDICATEUR_DEVDUR': 4,
 'len_TIME_PERIOD': 51}

Afin de pouvoir automatiser le traitement de manière à le rendre accessible aux métiers, il est nécessaire de produire des 'tableaux' propres contenant les modalités de chaque variable sur lesquelles se baseront les analyses par la suite. 

Je produis donc ci dessous des listes grâce aux formules Excel UNIQUE() et TRIER(). Les données ont précédemment été traitées pour ne plus apparaître sous forme de libellé codé, mais bien sous une forme explicite compréhensible et lisible pour les utilisateurs.

In [8]:
# Création d'une feuille d'index regroupant les valeurs de chaque colonne: 
## TODO >> Enlever les lignes de titre des listes produites (skip) // Avoir les libellés plutot que les codes dans les listes 

if 'INDEX' not in wb.sheetnames:
    index = wb.create_sheet('INDEX')
else:
    index = wb['INDEX']

# On remplit la feuille index à l'aide de formules =TRIER(UNIQUE()) et de l'extension ArrayFormula() qui permet d'afficher des résultats en liste:
formula = "=_xlfn.SORT(_xlfn.UNIQUE(DATA!A:A))"
index['A1'] = "Liste des ODD"
index['A2'] = ArrayFormula(f"A2:A{len_col['len_ODD']}", formula) #ODD

formula = "=_xlfn.SORT(_xlfn.UNIQUE(DATA!B:B))"
index['B1'] = "Liste des Indicateurs de DD"
index['B2'] = ArrayFormula(f"B2:B{len_col['len_INDICATEUR_DEVDUR']}", formula) #Indicateurs

formula = "=_xlfn.SORT(_xlfn.UNIQUE(DATA!H:H))"
index['C1'] = "Liste des Cibles de DD"
index['C2'] = ArrayFormula(f"C2:C{len_col['len_DEVDUR_COMPOSITE']}", formula) #Cibles

formula = "=_xlfn.SORT(_xlfn.UNIQUE(DATA!N:N))"
index['D1'] = "Année"
index['D2'] = ArrayFormula(f"D2:D{len_col['len_TIME_PERIOD']}", formula) #Années

wb.save(path_file)
wb.close()

Evolution des actions menées sur les ODD: 
1- Tbleau -- Formule: nb.si(time_period = time_period)
2- Graohique -- utiliser le tableau créé pour avoir l'évolution des ODD dans le temps. 

In [17]:

bilan['C5'] = '=COUNTIF(DATA!N:N, B5)'
bilan['B5'] = "2020"

bilan['C6'] = '=COUNTIF(DATA!N:N, B6)'
bilan['B6'] = "2021"

bilan['C7'] = '=COUNTIF(DATA!N:N, B7)'
bilan['B7'] = "2022"

bilan['C8'] = '=COUNTIF(DATA!N:N, B8)'
bilan['B8'] = "2023"

wb.save(path_file)
wb.close



<bound method Workbook.close of <openpyxl.workbook.workbook.Workbook object at 0x7f67a47e3ce0>>

Ces listes créées, je génère les filtres qui détermineront le périmètre d'analyse de l'affichage du tableau de bord 'BILAN'. 

Filtres: 
Année: TIME_PERIOD
ODD: ODD
Localisation: GEO

In [12]:
wb = load_workbook(path_file)

# Ré initialisation des éléments ( TODO à retirer si nn nécessaire)
if 'BILAN' not in wb.sheetnames:
       bilan = wb.create_sheet('BILAN')

index = wb['INDEX']
bilan = wb['BILAN']

# Retirer la grille ( TODO à retirer si nn nécessaire)
# bilan.sheet_view.showGridLines = False 

# Filtre pour les années
titre_f1 = bilan['A1']
titre_f1.value = 'Année'
titre_f1.alignment  = Alignment(horizontal='center', vertical='center')
titre_f1.fill = PatternFill(start_color='00C0C0C0', end_color='00C0C0C0', fill_type='solid')

bilan.merge_cells(start_row=1, 
                       start_column=1, 
                       end_row=2, 
                       end_column=2)

val_F1 = bilan['C1']
val_F1.value = 2020
val_F1.alignment  = Alignment(horizontal='center', vertical='center')
val_F1.fill = PatternFill(start_color='00C0C0C0', end_color='00C0C0C0', fill_type='solid')


bilan.merge_cells(start_row=3, 
                       start_column=1,
                       end_row=4,
                       end_column=2)

formula = f"='INDEX'!$D$1:$D${len_col['len_TIME_PERIOD']}"

#Ajouter la liste de validation pour créer le filtre
dv = DataValidation(type='list', formula1=formula)
bilan.add_data_validation(dv)
coord_filter_year = 'C1'
dv.add(coord_filter_year)

wb.save(path_file)
wb.close()

In [ ]:
"""
    Ajoute un filtre:
    - avec liste déroulante sur la liste des valeurs
    - titre,
    - valeur par défaut

    Paramètres :
    - sheet : Feuille Excel à laquelle ajouter le filtre.
    - title : Titre du filtre.
    - value : Valeur par défaut du filtre.
    - start_row : Ligne de début pour le titre du filtre.
    - start_column : Colonne de début pour le titre du filtre.
    - end_row : Ligne de fin pour le titre du filtre.
    - end_column : Colonne de fin pour le titre du filtre.
    - formula : Formule pour la liste de validation.
    """

    """
    Ajoute un titre de filtre à une feuille Excel et fusionne les cellules correspondantes.

    Cette fonction insère un texte (le titre du filtre) dans une plage de cellules spécifiée, 
    centre le texte horizontalement et verticalement, applique un fond gris clair, 
    et fusionne les cellules de la plage définie.

    Paramètres :
    - sheet: Feuille Excel dans laquelle ajouter le titre.
    - title: Texte à afficher comme titre du filtre.
    - start_row: Ligne de début de la plage à fusionner.
    - start_column: Colonne de début de la plage à fusionner.
    - end_row: Ligne de fin de la plage à fusionner.
    - end_column: Colonne de fin de la plage à fusionner.
    """

In [179]:
## On crée des fonctions imbriquées permettant de séparer la création de filtre par étape pour:
    # Simplifier la gestion
    # Simplifier les évolutions
    # Maximiser l'adaptabilité et la reproductibilité

def filter (sheet, row, col, title, row_dval, col_dval, col_filter_var, len_filter_var):
    filter_title = sheet.cell(row, col)
    filter_title.value = f"{title}"

    filter_default_value = sheet.cell(row_dval, col_dval)
    filter_default_value.value = "Tout"

    formula = f"='INDEX'!${col_filter_var}$2:${col_filter_var}${len_filter_var}"

    filter_values = DataValidation(
        type="list",
        formula1=formula,
        allow_blank=True
        )
    sheet.add_data_validation(dv)
    dv.add(sheet["G2"])

filter(bilan, 5, 7, "Année", 6, 7, 14, "{len_col['len_TIME_PERIOD']}")

BROUILLON FONCTION FORMULE


def formules(ws, target, sheet, col, range=None):
    """
    Insère dans une cellule une formule Excel qui extrait les valeurs uniques triées
    depuis une colonne donnée d'une autre feuille.
       ** ws : nom de la feuille résultat
       ** target : cellule cible du résultat
       ** sheet : nom de la feuille source
       ** col : colonne source 
       ** range : plage à sonder
       ** formule : la formule Excel voulue
    """
    if range:
        ref_range = f"{sheet}!{range}"
    else:
        ref_range = f"{sheet}!{col}:{col}"

    sort_unique = f"=_xlfn.SORT(_xlfn.UNIQUE({range})"
    ws[target] = ArrayFormula(f"{ref_range}", sort_unique)
